In [1]:
import torch

from torchvision import datasets
from torchvision import transforms
from torch import nn, optim
import timm
from timm.loss import LabelSmoothingCrossEntropy
import matplotlib.pyplot as plt
%matplotlib inline

import sys
from tqdm import tqdm
import time
import copy
from torchvision import datasets, transforms, models
import torch
import torchvision

import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F

from torchvision.transforms import ToTensor
from torchvision.utils import make_grid
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
normal_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225]
    )
])

data_dir = r'../data/crop/images/'
dataset = datasets.ImageFolder(data_dir, transform=normal_transforms)

train_size = int(0.7 * len(dataset))
validate_size = int(0.15 * len(dataset))
test_size = len(dataset) - train_size - validate_size

train_dataset, validate_dataset, test_dataset = random_split(dataset, [train_size, validate_size, test_size])

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True, pin_memory=True)
validate_loader = torch.utils.data.DataLoader(validate_dataset, batch_size=32, shuffle=True, pin_memory=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=True, pin_memory=True)

In [5]:
dataloaders = {
    'train': train_loader,
    'validate': validate_loader
}
dataset_sizes = {
    'train': train_size,
    'validate': validate_size
}

In [6]:
def get_classes(data_dir):
    all_data = datasets.ImageFolder(data_dir)
    return all_data.classes

classes = get_classes(data_dir)

In [7]:
model = torch.load(r'../model/demo_resnet18')

In [27]:
from PIL import Image

image_path = r'../data/crop/images/Nanowires/L6_00256e4ab29cc0903c09269165f96443.jpg'
image = Image.open(image_path)
input_image = normal_transforms(image)
input_image = input_image.unsqueeze(0)

with torch.no_grad():
    output = model(input_image.cuda())

_, predicted_label = torch.max(output, 1)

print('label : ', predicted_label)

label :  tensor([4], device='cuda:0')


In [28]:
import torch
from torchvision import transforms, datasets
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

model.to(device)
model.eval()

true_labels = []
predicted_labels = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)

        true_labels.extend(labels.cpu().numpy())
        predicted_labels.extend(preds.cpu().numpy())

conf_matrix = confusion_matrix(true_labels, predicted_labels)

accuracy = accuracy_score(true_labels, predicted_labels)

precision = precision_score(true_labels, predicted_labels, average='weighted')

recall = recall_score(true_labels, predicted_labels, average='weighted')

f1 = f1_score(true_labels, predicted_labels, average='weighted')

print(f'Confusion Matrix:\n{conf_matrix}')
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')
        

Confusion Matrix:
[[132   0   0   2   9   4   1   0   1   0]
 [  2   3   0   5   5   0   0   0   0   1]
 [  0   0  13   2   4  22   2   0   7   2]
 [  2   0   0 570   7   4  36   0   0  17]
 [  3   0   0  11 457  46   5   0   3  20]
 [  0   0   1   1   3 475   6   1   4   3]
 [  2   1   1  35  11  25 401   2   1  19]
 [  1   0   0   2   3   4   1  10   0   0]
 [  1   0   0   0  12  12   3   0 114   1]
 [  2   0   0   8   2   6  13   0   0 203]]
Accuracy: 0.8529411764705882
Precision: 0.8563285848867715
Recall: 0.8529411764705882
F1 Score: 0.8481356393827688
